In [1]:
#importing libraries
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.82190895080566

Epoch: 2 	Loss: 428.685409784317

Epoch: 3 	Loss: 425.05888843536377

Epoch: 4 	Loss: 421.83957743644714

Epoch: 5 	Loss: 419.5340943336487



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.008117,0.043179,-0.025967,-0.053352,0.064635,0.002694,-0.019058,0.011148,-0.001328,0.011125,...,0.008347,-0.010539,0.060543,-0.015804,-0.002581,0.007230,0.008506,0.033016,-0.002336,0.025570
networks,-0.030135,0.005777,0.015104,0.031092,-0.003619,0.032982,0.055717,-0.016277,0.008634,-0.011236,...,0.007576,-0.008316,-0.014788,0.031822,0.059783,0.000668,0.001520,0.023409,0.012868,-0.017669
neural,-0.006061,0.032205,0.030097,-0.025263,0.015152,0.038714,0.035798,0.038655,0.047493,0.000686,...,-0.001868,-0.006777,0.028116,-0.014916,0.034036,-0.010436,0.023688,-0.034369,-0.043410,-0.049589
and,0.012080,-0.028633,0.040001,-0.043334,-0.030806,0.018179,-0.035860,-0.001465,0.009119,-0.010700,...,0.014389,0.029863,0.017514,-0.030153,0.020659,-0.031148,0.036823,0.002808,0.015028,0.011079
as,-0.014432,0.010052,0.011666,0.017830,0.015983,0.005682,0.048186,0.020674,0.016271,0.044427,...,0.037306,-0.042479,0.036177,-0.025181,-0.003661,-0.009195,0.022259,0.028194,-0.049714,-0.012457


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['family', 'recurrent', 'artificial', 'networks', 'surpassing']}

In [11]:
f = open("/Users/lenovo/Desktop/demo.txt", "r")
print(f.read())

Hello! Welcome to demofile.txt
This file is for testing purposes.
Good Luck!s


In [14]:
with open('/Users/lenovo/Desktop/demo.txt', 'r') as file:
    data1 = file.read().replace('\n', '')

In [15]:
dl_data1 = data1.split()

In [16]:
print(dl_data1)

['Hello!', 'Welcome', 'to', 'demofile.txtThis', 'file', 'is', 'for', 'testing', 'purposes.Good', 'Luck!s']


In [17]:
print(data1)

Hello! Welcome to demofile.txtThis file is for testing purposes.Good Luck!s
